---

<div align=center>

# **Initialisation**

</div>

### Dependencies

In [1]:
%%capture

import numpy as np
import tensorflow as tf
from LRFutils import archive, progress
import os
from multiprocess import Pool, cpu_count
import psutil
import pandas as pd

2023-03-20 16:41:15.809682: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 16:41:16.545334: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-20 16:41:16.545370: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-20 16:42:29.896606: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

### Configuration

In [2]:
valid_frac = 0.2
test_frac  = 0.1
dataset_path = "data/dataset"

### Global variables

In [3]:
archive_path = archive.new()

try:
    ncpu = cpu_count()
except:
    with open(os.getenv("OAR_NODEFILE"), 'r') as f:
        ncpu = len(f.readlines())

### Useful functions

In [4]:
def system_info():
    return f"CPU: {psutil.cpu_percent()}%"\
        + f", RAM: {psutil.virtual_memory().percent}%"\
        + f" ({psutil.virtual_memory().used/1024**3:.2f}GB"\
        + f"/ {psutil.virtual_memory().total/1024**3:.2f}GB)"

---

<div align=center>

# **Load data**

</div>

### Read one file

In [5]:
def load_file(file):

    data = np.load(file)

    x = [
        250, # dust observation frequency [um}
        data["dust_image"],
        data["CO_v"],
        data["CO_cube"],
        data["N2H_v"],
        data["N2H_cube"]
    ]

    y = data["mass"]

    return x, y

### Read dataset

In [6]:
def load_data() -> tuple[np.ndarray, tuple[np.ndarray, np.ndarray, np.ndarray]]:
    """Do what you want int this function, as long as it returns the following:
    - list[3D-ndarray] : input vectors
    - list[3D-ndarray] : output vectors
    """

    # Limit of the number of vectors to load
    max_files = 1000
    files = os.listdir(dataset_path)
    nb_vectors = min(len(files), max_files)

    # Load data
    x = []
    y = []
    bar = progress.Bar(nb_vectors, "Loading data")
    for i, file in enumerate(files):
        if i >= nb_vectors:
            break
        bar(i, prefix=system_info())
        
        new_x, new_y = load_file(f"{dataset_path}/{file}")
        x.append(new_x)
        y.append(new_y)
        
    bar(nb_vectors)    
    return x, y

In [7]:
x, y = load_data()
nb_vectors = len(x)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 1000/1000 0:00:577 eta 0:00:00m


### Represenation of the data

In [11]:
df = pd.DataFrame(x)
df.columns = ["Dust observation wavelength", "Dust map", "CO velocity", "CO cube", "N2H velocity", "N2H cube"]
df

,Dust observation wavelength,Dust map,CO velocity,CO cube,N2H velocity,N2H cube
0,250,"[[19651.43, 12827.988, 12558.087, 12461.606, 1...","[-0.49609375, -0.48828125, -0.48046875, -0.472...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-0.49609375, -0.48828125, -0.48046875, -0.472...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
1,250,"[[14849.689, 10940.576, 10670.075, 10766.056, ...","[-0.49609375, -0.48828125, -0.48046875, -0.472...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-0.49609375, -0.48828125, -0.48046875, -0.472...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,250,"[[207.65686, 76.80976, 70.43741, 65.994, 63.78...","[-0.49609375, -0.48828125, -0.48046875, -0.472...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-0.49609375, -0.48828125, -0.48046875, -0.472...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
3,250,"[[118.43301, 45.763374, 45.159184, 39.03437, 3...","[-0.49609375, -0.48828125, -0.48046875, -0.472...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-0.49609375, -0.48828125, -0.48046875, -0.472...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,250,"[[79.88295, 30.143417, 28.508665, 26.439205, 2...","[-0.49609375, -0.48828125, -0.48046875, -0.472...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-0.49609375, -0.48828125, -0.48046875, -0.472...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
...,...,...,...,...,...,...
995,250,"[[609.69135, 224.63101, 210.59906, 192.80165, ...","[-0.49609375, -0.48828125, -0.48046875, -0.472...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-0.49609375, -0.48828125, -0.48046875, -0.472...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
996,250,"[[3934.8936, 1433.1036, 1357.4531, 1280.2007, ...","[-0.49609375, -0.48828125, -0.48046875, -0.472...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-0.49609375, -0.48828125, -0.48046875, -0.472...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
997,250,"[[8.802127, 2.9607053, 2.7406383, 2.469004, 2....","[-0.49609375, -0.48828125, -0.48046875, -0.472...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-0.49609375, -0.48828125, -0.48046875, -0.472...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
998,250,"[[1.460321, 0.48044378, 0.4456209, 0.41084996,...","[-0.49609375, -0.48828125, -0.48046875, -0.472...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-0.49609375, -0.48828125, -0.48046875, -0.472...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."


---

<div align=center>

# **Post processing**

</div>

### Normalisation

In [12]:
maximums = [
    x[0][0],
    x[0][1][0,0],
    x[0][2][0],
    x[0][3][0,0,0],
    x[0][4][0],
    x[0][5][0,0,0]
]

for vector in x:
    for i in range(len(vector)):
        if (value := np.max(np.abs(vector[i]))) > maximums[i]:
            maximums[i] = value

for vector in x:
    for i in range(len(vector)):
        vector[i] /= maximums[i]

print(maximums)

[250, 22901.021, 0.49609375, 6.65215, 0.49609375, 7.187931]


### Splitting datasets
#### Train

In [13]:
train_frac = 1 - valid_frac - test_frac

train_x = x[:int(nb_vectors * train_frac)]
train_y = y[:int(nb_vectors * train_frac)]

df = pd.DataFrame(train_x)
df.columns = ["Dust observation wavelength", "Dust map", "CO velocity", "CO cube", "N2H velocity", "N2H cube"]
df

,Dust observation wavelength,Dust map,CO velocity,CO cube,N2H velocity,N2H cube
0,1.0,"[[0.85810274, 0.5601492, 0.5483636, 0.5441507,...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
1,1.0,"[[0.64842916, 0.4777331, 0.46592137, 0.4701124...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,1.0,"[[0.009067581, 0.0033539885, 0.0030757322, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
3,1.0,"[[0.0051715164, 0.0019983114, 0.0019719289, 0....","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,1.0,"[[0.0034881828, 0.0013162477, 0.0012448643, 0....","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
...,...,...,...,...,...,...
695,1.0,"[[0.5239278, 0.40644476, 0.40430132, 0.4108038...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
696,1.0,"[[0.00047551212, 0.00017605844, 0.00015870904,...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
697,1.0,"[[3.8795864e-05, 1.261611e-05, 1.1615677e-05, ...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
698,1.0,"[[0.0025198932, 0.0009794647, 0.00089673576, 0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."


#### Validation

In [14]:
valid_x = x[int(nb_vectors * train_frac):int(nb_vectors * (train_frac + valid_frac))]
valid_y = y[int(nb_vectors * train_frac):int(nb_vectors * (train_frac + valid_frac))]

df = pd.DataFrame(valid_x)
df.columns = ["Dust observation wavelength", "Dust map", "CO velocity", "CO cube", "N2H velocity", "N2H cube"]
df

,Dust observation wavelength,Dust map,CO velocity,CO cube,N2H velocity,N2H cube
0,1.0,"[[0.93085766, 0.47837305, 0.45591435, 0.444176...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
1,1.0,"[[0.0015421521, 0.00058503076, 0.00056996604, ...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,1.0,"[[0.00023069633, 7.831815e-05, 7.035095e-05, 6...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
3,1.0,"[[0.0029396273, 0.0011451751, 0.0010763599, 0....","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,1.0,"[[0.00012905756, 0.00013943923, 0.00015065285,...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
...,...,...,...,...,...,...
195,1.0,"[[0.3328758, 0.28472397, 0.2811066, 0.28617498...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
196,1.0,"[[0.040049203, 0.014533083, 0.0136056645, 0.01...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
197,1.0,"[[1.3153868e-08, 1.4368542e-08, 1.5713088e-08,...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
198,1.0,"[[9.710688e-05, 3.389518e-05, 3.0365542e-05, 2...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."


#### Test

In [15]:
test_x = x[int(nb_vectors * (train_frac + valid_frac)):]
test_y = y[int(nb_vectors * (train_frac + valid_frac)):]

df = pd.DataFrame(test_x)
df.columns = ["Dust observation wavelength", "Dust map", "CO velocity", "CO cube", "N2H velocity", "N2H cube"]
df

,Dust observation wavelength,Dust map,CO velocity,CO cube,N2H velocity,N2H cube
0,1.0,"[[0.000629175, 0.00023543218, 0.00021238529, 0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
1,1.0,"[[0.059716105, 0.060401034, 0.06296737, 0.0661...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,1.0,"[[0.0012280368, 0.0004591296, 0.00044194207, 0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
3,1.0,"[[8.994029e-06, 9.793927e-06, 1.0654216e-05, 1...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,1.0,"[[0.005317555, 0.0019609912, 0.0019534226, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
...,...,...,...,...,...,...
95,1.0,"[[0.02662289, 0.009808777, 0.009196055, 0.0084...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
96,1.0,"[[0.17182174, 0.06257816, 0.059274785, 0.05590...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
97,1.0,"[[0.0003843552, 0.00012928268, 0.00011967319, ...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
98,1.0,"[[6.376663e-05, 2.0979141e-05, 1.945856e-05, 1...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[-1.0, -0.984251968503937, -0.968503937007874,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."


---

<div align=center>

# **Model definition**

</div>

### Build model

In [16]:
def get_model(input_shape, output_shape):
    from keras.layers import Input, Dense, Conv2D, MaxPooling2D, MaxPooling3D, UpSampling2D, UpSampling3D, Reshape, Conv3DTranspose, Flatten
    from keras.models import Model

    # Définir la forme de l'image d'entrée
    input = Input(shape=(64, 64, 1))

    # Encoder
    x = Conv2D(32, (15, 15), activation='relu', padding='same')(input)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (5, 5), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    output = Dense(1, activation='relu')(x)

    # Modèle d'auto-encodeur
    model = Model(input, output)

    return model

### Compile model and get summary

In [18]:
model = get_model(x[0][1].shape, np.array(y[0]).shape)
# model.compile(optimizer='adam', loss='binary_crossentropy')
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.summary()

2023-03-20 16:56:56.345846: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-20 16:56:56.345883: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-20 16:56:56.345909: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cluster16): /proc/driver/nvidia/version does not exist
2023-03-20 16:56:56.346482: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 32)        7232      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 16384)             0     

### Human validation of the model

In [ ]:
choice = input("Continue ? [Y/n]")

if choice.lower() not in ["", "y", "yes"]:
    exit()

### Training model

In [ ]:
model.fit(train_x, train_y, epochs=10, batch_size=32, validation_data=(valid_x, valid_y))
model.save(f'{archive_path}/model0.h5')

### Evaluating model

In [ ]:
score = model.evaluate(test_x, test_y, verbose=0)
print("Score:", score)

with open(f'{archive_path}/scores.txt', 'w') as f:
    f.write(f'Score:    \t{score}\n')

---

<div align=center>

# Predictions

</div>

In [ ]:
x_prediction = [x[0,...]]
x_prediction = np.expand_dims(x_prediction, axis=-1)
print(x_prediction.shape)

y_prediction = model.predict(x_prediction)
print(y_prediction.shape)

np.savez_compressed(f'{archive_path}/prediction.npz', x=x_prediction, y=y_prediction)